# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [1]:
#!pip install -qU ragas==0.2.10

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

NLTK is a standard Python library for natural language processing—it bundles ready-to-use, pretrained tools for splitting text (tokenization), tagging parts of speech, parsing, classification, and more.

Consistency across platforms: Punkt handles edge cases (abbreviations, punctuation) uniformly on Windows, Linux, macOS, etc., avoiding locale- or OS-specific quirks.

In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/sa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/sa/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

This sets an environment variable that enables LangChain's advanced tracing system (v2).
This tracing captures detailed info about each step in your pipeline (like prompt calls, LLM usage, vector store queries) and sends it to LangSmith or other backends for debugging, performance monitoring, and auditing.
Without it, tracing stays off and no execution data is collected.

In [4]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [5]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [6]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [7]:
# 1
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [8]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())
# 2

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [9]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg
# 3

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [10]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg
# 4

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [11]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg
# 5

Applying HeadlinesExtractor:   0%|          | 0/14 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/25 [00:00<?, ?it/s]

Property 'summary' already exists in node 'c36adb'. Skipping!
Property 'summary' already exists in node 'a188fc'. Skipping!
Property 'summary' already exists in node '4cb1af'. Skipping!
Property 'summary' already exists in node '3a317d'. Skipping!
Property 'summary' already exists in node 'd03155'. Skipping!
Property 'summary' already exists in node 'a40fea'. Skipping!
Property 'summary' already exists in node '25ccba'. Skipping!
Property 'summary' already exists in node '85396c'. Skipping!
Property 'summary' already exists in node '7708ed'. Skipping!
Property 'summary' already exists in node '3199a5'. Skipping!
Property 'summary' already exists in node '728c7b'. Skipping!


Applying CustomNodeFilter:   0%|          | 0/7 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/35 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node 'a188fc'. Skipping!
Property 'summary_embedding' already exists in node 'c36adb'. Skipping!
Property 'summary_embedding' already exists in node 'd03155'. Skipping!
Property 'summary_embedding' already exists in node 'a40fea'. Skipping!
Property 'summary_embedding' already exists in node '4cb1af'. Skipping!
Property 'summary_embedding' already exists in node '25ccba'. Skipping!
Property 'summary_embedding' already exists in node '3199a5'. Skipping!
Property 'summary_embedding' already exists in node '728c7b'. Skipping!
Property 'summary_embedding' already exists in node '3a317d'. Skipping!
Property 'summary_embedding' already exists in node '7708ed'. Skipping!
Property 'summary_embedding' already exists in node '85396c'. Skipping!


Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 36, relationships: 311)

We can save and load our knowledge graphs as follows.

In [12]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg
# 6

KnowledgeGraph(nodes: 36, relationships: 311)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [13]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)
# 7

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [14]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]
# 8


#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

#### ✅ Answer 

Here is a concise English summary of each query synthesizer:

---

**🔹 SingleHopSpecificQuerySynthesizer**

* **What it does**: Generates questions answerable using a single node from the graph.
* **How it’s formulated**: Uses exact details (names, numbers, dates) as they appear in the source.
* **Example**: “What was the average interest rate of loan X in March 2025?”

---

**🔹 MultiHopAbstractQuerySynthesizer**

* **What it does**: Creates questions that require connecting multiple nodes to answer.
* **How it’s formulated**: Uses conceptual or generalized phrasing, without quoting exact values.
* **Example**: “How are loan durations related to credit risk?”

---

**🔹 MultiHopSpecificQuerySynthesizer**

* **What it does**: Also requires multi-node reasoning, like the abstract variant.
* **How it’s formulated**: Includes specific facts (exact values, entity names).
* **Example**: “What’s the difference in default rates between loan A and loan B during the first half of 2025?”

---




Finally, we can use our `TestSetGenerator` to generate our testset!

In [15]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()
# 9 

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,Is the United States the country where Pell Gr...,[General Eligibility Requirements for Federal ...,The context discusses eligibility requirements...,single_hop_specifc_query_synthesizer
1,What happens if I report the wrong grade level...,[Wrong Grade Level on the FAFSA Form When an u...,If an undergraduate student incorrectly report...,single_hop_specifc_query_synthesizer
2,How does the Pell Grant eligibility work for s...,[Eligible Postbaccalaureate Program A student ...,A student enrolled at least half time in a pos...,single_hop_specifc_query_synthesizer
3,What are Postbaccalaureate Programs?,[Eligible Postbaccalaureate Programs 34 CFR 69...,Eligible Postbaccalaureate Programs are refere...,single_hop_specifc_query_synthesizer
4,Can you tell me what the FSA Handbook says abo...,[Determining Enrollment Intensity Using Credit...,The FSA Handbook explains that enrollment inte...,single_hop_specifc_query_synthesizer
5,Considering the eligibility requirements for F...,[<1-hop>\n\nGeneral Eligibility Requirements f...,Providing incorrect information about degree s...,multi_hop_abstract_query_synthesizer
6,Hwo do the R2T4 proceudres affect the program ...,[<1-hop>\n\nEligible Postbaccalaureate Program...,The R2T4 procedures impact students in postbac...,multi_hop_abstract_query_synthesizer
7,How do program requirements and regulatory ref...,[<1-hop>\n\nEligible Postbaccalaureate Program...,Program requirements specify that eligible pos...,multi_hop_abstract_query_synthesizer
8,Postbaccalaureate Programs Pell how get money ...,[<1-hop>\n\nEligible Postbaccalaureate Program...,If you enroll at least half time in a postbacc...,multi_hop_specific_query_synthesizer
9,"How does Pell relate to Pell, and how does Pel...",[<1-hop>\n\nDetermining Enrollment Intensity U...,The context discusses Pell in relation to enro...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [16]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)
# 10

Applying HeadlinesExtractor:   0%|          | 0/14 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/25 [00:00<?, ?it/s]

Property 'summary' already exists in node 'b464f6'. Skipping!
Property 'summary' already exists in node '9dbf66'. Skipping!
Property 'summary' already exists in node 'b577f4'. Skipping!
Property 'summary' already exists in node '887693'. Skipping!
Property 'summary' already exists in node '617f2e'. Skipping!
Property 'summary' already exists in node '75fb9f'. Skipping!
Property 'summary' already exists in node 'a8b554'. Skipping!
Property 'summary' already exists in node '1ae9b3'. Skipping!
Property 'summary' already exists in node '44d46c'. Skipping!
Property 'summary' already exists in node 'e73390'. Skipping!
Property 'summary' already exists in node 'a37752'. Skipping!


...

Property 'summary_embedding' already exists in node 'b464f6'. Skipping!
Property 'summary_embedding' already exists in node '9dbf66'. Skipping!
Property 'summary_embedding' already exists in node 'a37752'. Skipping!
Property 'summary_embedding' already exists in node '75fb9f'. Skipping!
Property 'summary_embedding' already exists in node 'b577f4'. Skipping!
Property 'summary_embedding' already exists in node '1ae9b3'. Skipping!
Property 'summary_embedding' already exists in node '617f2e'. Skipping!
Property 'summary_embedding' already exists in node '44d46c'. Skipping!
Property 'summary_embedding' already exists in node '887693'. Skipping!
Property 'summary_embedding' already exists in node 'a8b554'. Skipping!
Property 'summary_embedding' already exists in node 'e73390'. Skipping!


Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [17]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Federal Pell Grants eligibility?,[General Eligibility Requirements for Federal ...,Federal Pell Grants are generally available to...,single_hop_specifc_query_synthesizer
1,Federal Student Aid what happen if wrong grade...,[Wrong Grade Level on the FAFSA Form When an u...,If an undergraduate student reports incorrectl...,single_hop_specifc_query_synthesizer
2,Is the Federal Supplemental Educational Opport...,[Eligible Postbaccalaureate Program A student ...,"No, students enrolled in a postbaccalaureate t...",single_hop_specifc_query_synthesizer
3,What does HEA Sec. 401(d)(4)(B) mean for stude...,[Eligible Postbaccalaureate Programs 34 CFR 69...,HEA Sec. 401(d)(4)(B) relates to the prohibiti...,single_hop_specifc_query_synthesizer
4,Wht is the requirments for Pell Grant eligibil...,[<1-hop>\n\nGeneral Eligibility Requirements f...,The eligibility for a Federal Pell Grant depen...,multi_hop_abstract_query_synthesizer
5,How do restrictions on concurrent Pell Grant a...,[<1-hop>\n\nEligible Postbaccalaureate Program...,Students enrolled in eligible postbaccalaureat...,multi_hop_abstract_query_synthesizer
6,How does documentation ensure compliance for P...,[<1-hop>\n\nEligible Postbaccalaureate Program...,Documentation ensures compliance for Pell Gran...,multi_hop_abstract_query_synthesizer
7,"So, like, how does enrollment intensity, which...",[<1-hop>\n\nDetermining Enrollment Intensity U...,The context explains that enrollment intensity...,multi_hop_abstract_query_synthesizer
8,Wht Pell is inccorect on FAFSA if grad or bach?,[<1-hop>\n\nWrong Grade Level on the FAFSA For...,If an undergraduate student reports on the FAF...,multi_hop_specific_query_synthesizer
9,title iv student get pell grant if in postbacc...,[<1-hop>\n\nEligible Postbaccalaureate Program...,A student in an eligible postbaccalaureate pro...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [18]:
from langsmith import Client

client = Client()

dataset_name = "Loan Synthetic Data"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data"
)
# 11

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [19]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )
  # 12

## Basic RAG Chain

Time for some RAG!


In [20]:
rag_documents = docs
# 13

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)
# 13

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [22]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
# 13

As usual, we will power our RAG application with Qdrant!

In [23]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)
# 13

In [24]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
# 13

To get the "A" in RAG, we'll provide a prompt.

In [25]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)
# 13

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [26]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")
# 13

Finally, we can set-up our RAG LCEL chain!

In [27]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)
# 14

In [28]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})
# 14

'The kinds of loans available include:\n\n- Direct Subsidized Loans  \n- Direct Unsubsidized Loans  \n- Direct PLUS Loans (student Federal PLUS Loans or parent PLUS Loans)  \n- Subsidized and Unsubsidized Federal Stafford Loans  \n- Federal SLS Loans  \n- Federal PLUS Loans  \n\nNote: Subsidized and Unsubsidized Federal Stafford Loans, Federal SLS Loans, and Federal PLUS Loans were made under the Federal Family Education Loan (FFEL) Program, which ended new loan issuance effective July 1, 2010. New loans under the Direct Loan Program include Direct Subsidized, Direct Unsubsidized, and Direct PLUS Loans.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [29]:
eval_llm = ChatOpenAI(model="gpt-4.1")
# 15

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [30]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
        
    }
)
# 15

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

#### ✅ Answer

- `qa_evaluator`: Checks factual correctness, whether the generated answer matches the reference answer.
- `labeled_helpfulness_evaluator`: Assesses helpfulness, judging if the submission effectively helps the user given the correct reference.
- `empathy_evaluator`: Measures empathy, determining whether the response is kind, supportive, and makes the user feel heard

## LangSmith Evaluation

In [31]:
evaluate(
    rag_chain.invoke,
    data=0,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)
# 16

View the evaluation results for experiment: 'yellow-instrument-12' at:
https://smith.langchain.com/o/6aa39193-2c1d-4f43-b4b1-8faaa8260038/datasets/ad26d8b5-84c0-4f45-94c1-ee00157f9295/compare?selectedSessions=475c556f-336e-4c77-8532-d5594b6eae84




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,what is the probhition on conccurent enrollmne...,The prohibition on concurrent enrollment in el...,None,According to 34 CFR 690.6(c) and HEA Sec. 401(...,1,1,0,2.953546,d7fd3969-6570-408d-a1ae-7b17476004e8,ce08f500-d537-4b1b-b8f5-945b46a0ddee
1,How does correctly reporting a bachelor's degr...,Correctly reporting that the student has a bac...,None,Correctly reporting a bachelor's degree on the...,1,1,0,4.758844,7532182f-bc34-4ea9-bad6-d6c00aa9a990,4384e4c3-7a6b-4ad1-be7f-90662da1b249
2,title iv student get pell grant if in postbacc...,Yes. Students who correctly report that they h...,None,A student in an eligible postbaccalaureate pro...,1,1,0,4.227446,edbf4d63-bb50-455b-8fb2-843c37e8ec80,40ca7fdb-1bfb-42ba-9b95-f73a032f4961
3,Wht Pell is inccorect on FAFSA if grad or bach?,Pell Grant eligibility is incorrect on the FAF...,None,If an undergraduate student reports on the FAF...,1,1,0,2.463280,b4a30f5d-424b-4d94-a5d7-b50fac2a5271,9b3f656c-1ae0-4f10-a3a7-ef9baf28cb50
4,"So, like, how does enrollment intensity, which...",Based on the provided context:\n\nEnrollment i...,None,The context explains that enrollment intensity...,1,1,0,6.310033,452777ec-6c66-4818-a18b-37f13a0c865a,3ef93edd-62af-4cc8-9834-a3d0a476c282
5,How does documentation ensure compliance for P...,"Based on the provided context, documentation i...",None,Documentation ensures compliance for Pell Gran...,1,1,0,7.766982,4e1f1944-0c88-47d3-a512-379bb634fe68,761760a6-0170-4040-ac95-b6c60a35803e
6,How do restrictions on concurrent Pell Grant a...,Based on the provided context:\n\nStudents enr...,None,Students enrolled in eligible postbaccalaureat...,1,0,0,5.003045,d3820eee-fa6f-474a-a614-6da63e763f3b,2facec04-3f71-4c43-926a-b87f49a817e4
7,Wht is the requirments for Pell Grant eligibil...,Based on the provided context:\n\n**Requiremen...,None,The eligibility for a Federal Pell Grant depen...,1,1,0,5.610864,fc30bee0-c320-4834-9a97-24f11d3bc8cc,cc632a9a-0c59-434a-b4c5-3b1530d48fe4
8,What does HEA Sec. 401(d)(4)(B) mean for stude...,HEA Sec. 401(d)(4)(B) relates to the definitio...,None,HEA Sec. 401(d)(4)(B) relates to the prohibiti...,0,0,0,3.908874,527bda40-4409-4596-9f20-60bc58039605,698c19ab-5377-416a-8066-377beee4107b
9,Is the Federal Supplemental Educational Opport...,"Based on the provided context, a student enrol...",None,"No, students enrolled in a postbaccalaureate t...",1,1,0,2.237516,934ccae6-073e-4d27-82c5-1897cd61ef35,75d1a6f2-b352-4476-866d-4cfb660ac90e


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [ ]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)
# 17

In [33]:
rag_documents = docs

In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

#### ✅ Answer 

Modifying chunk size changes how your application breaks up and retrieves information, which directly affects both **accuracy** and **efficiency**:

* **Smaller chunks** improve **retrieval precision**. It's easier to find the exact relevant piece of text, 
but increase the number of embeddings, making indexing and querying slower and more expensive.
* **Larger chunks** reduce index size and speed up retrieval, but may include **irrelevant context**, reducing answer quality and increasing token usage in prompts.

So, chunk size impacts **latency, cost, and response accuracy**, making it a critical performance factor in RAG pipelines.


In [35]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

#### ✅ Answer

Modifying your embedding model affects performance because it changes how your application understands and represents text for retrieval. This impacts both accuracy and speed:

Semantic quality: A more powerful model captures deeper meaning and context, improving retrieval accuracy and response quality. A weaker model may miss important nuances, leading to irrelevant or incomplete results.

Token handling: Different models support different max token limits per input. A model with higher capacity can handle longer chunks, reducing fragmentation and preserving more context.

Latency and cost: Larger, more accurate models are often slower and more expensive to run. Lightweight models are faster and cheaper but may trade off retrieval precision.

In [36]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [37]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [38]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [39]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

"Thank you for your question—it's important to understand the types of loans that might be available to help you with educational expenses. Based on the information provided, there are several kinds of loans you can consider:\n\n1. **Direct Subsidized Loans**: These loans are based on the student's financial need. Interest is paid by the government while the student is in school at least half-time.\n\n2. **Direct Unsubsidized Loans**: These loans are not need-based, and interest accrues during all periods.\n\n3. **Direct PLUS Loans**: These can be taken out by the parents of dependent students to help cover the cost of attendance, provided the parent meets eligibility requirements. If the parent is ineligible or the student is independent, additional Direct Unsubsidized Loan amounts are available to the student.\n\nIt's also mentioned that some students may qualify for a combination of both Direct Subsidized and Direct Unsubsidized Loans to cover unmet financial need.\n\nI hope this he

Finally, we can evaluate the new chain on the same test set!

In [40]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'reflecting-stop-12' at:
https://smith.langchain.com/o/6aa39193-2c1d-4f43-b4b1-8faaa8260038/datasets/ad26d8b5-84c0-4f45-94c1-ee00157f9295/compare?selectedSessions=a76d1d6b-4a24-44e1-93e3-b5a85202439a




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,what is the probhition on conccurent enrollmne...,Thank you for your thoughtful question about t...,None,According to 34 CFR 690.6(c) and HEA Sec. 401(...,0,0,1,4.785022,d7fd3969-6570-408d-a1ae-7b17476004e8,db35e6d5-a793-424c-bf39-55bc6da84740
1,How does correctly reporting a bachelor's degr...,Thank you for your thoughtful question. Based ...,None,Correctly reporting a bachelor's degree on the...,1,1,1,4.315061,7532182f-bc34-4ea9-bad6-d6c00aa9a990,9bf60c46-d76a-4d7b-bb87-f565e7c1af99
2,title iv student get pell grant if in postbacc...,I understand that navigating Pell Grant eligib...,None,A student in an eligible postbaccalaureate pro...,1,1,1,3.976645,edbf4d63-bb50-455b-8fb2-843c37e8ec80,a41b4df9-574f-4b97-89a5-4b2811d0ed98
3,Wht Pell is inccorect on FAFSA if grad or bach?,I understand how confusing it can be to naviga...,None,If an undergraduate student reports on the FAF...,1,1,1,3.998651,b4a30f5d-424b-4d94-a5d7-b50fac2a5271,b0b8554d-e418-482a-9296-98400f99b3ff
4,"So, like, how does enrollment intensity, which...",Thank you for your thoughtful question—it's cl...,None,The context explains that enrollment intensity...,1,1,1,8.264623,452777ec-6c66-4818-a18b-37f13a0c865a,2c79cab9-e19e-4250-8055-62ab3c8240e6
5,How does documentation ensure compliance for P...,Thank you for your thoughtful question about h...,None,Documentation ensures compliance for Pell Gran...,1,0,1,5.891694,4e1f1944-0c88-47d3-a512-379bb634fe68,f4ecef5c-90ba-463a-82af-6108bfece7c8
6,How do restrictions on concurrent Pell Grant a...,Thank you for sharing your question—it's very ...,None,Students enrolled in eligible postbaccalaureat...,1,1,1,7.775299,d3820eee-fa6f-474a-a614-6da63e763f3b,54267b1a-693d-433c-a426-93025ca381cd
7,Wht is the requirments for Pell Grant eligibil...,Thank you for your thoughtful question about P...,None,The eligibility for a Federal Pell Grant depen...,1,0,1,6.780444,fc30bee0-c320-4834-9a97-24f11d3bc8cc,4c04f995-d0aa-4f4c-a9ce-f8b1d17a559f
8,What does HEA Sec. 401(d)(4)(B) mean for stude...,Thank you for your thoughtful question. Based ...,None,HEA Sec. 401(d)(4)(B) relates to the prohibiti...,1,1,1,5.698436,527bda40-4409-4596-9f20-60bc58039605,fcc8ef6e-d873-4b7a-82b9-590ec4685df5
9,Is the Federal Supplemental Educational Opport...,I understand how important it is to know exact...,None,"No, students enrolled in a postbaccalaureate t...",1,1,1,3.739920,934ccae6-073e-4d27-82c5-1897cd61ef35,1c18d093-1a8c-47da-9de0-55322485959b


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

# First Chain

![alt text](img/trace-1.png)

# Second Chain

![alt text](img/trace-2.png)


As we can see after “dope-ifying” the RAG chain by injecting empathetic instructions and adjusting chunk size some key metrics changed. Empathy rose from 0.0 to 1.0 thanks to the explicit prompt guiding the LLM to respond with kindness and understanding. However, helpfulness slightly dropped (from 0.8333 to 0.75), likely because the model added friendly but less informative language, reducing directness in some responses.

Latency increased (4.49s → 5.14s) due to larger chunk sizes and a longer prompt, which caused slower retrieval and longer LLM processing time. Despite this, total tokens and cost decreased significantly (39,689 → 23,635 tokens; $0.0188 → $0.0133), since fewer, larger chunks meant fewer embeddings and less overall data to process. This shows a trade-off between tone, speed, and cost-efficiency.

